#### This Module detects drowsiness in a driver by combining computer vision and unsupervised machine learning algorithms to detect and predict if the driver is drowsy or not. 

Libraries

In [2]:
import cv2
import mediapipe as mp
import numpy as np

Objects

In [3]:
# Drowsiness detection using face landmark detection
mp_sol = mp.solutions
mp_drawing = mp_sol.drawing_utils
mp_drawing_styles = mp_sol.drawing_styles
mp_face_mesh = mp_sol.face_mesh
facemesh = mp_face_mesh.FaceMesh(max_num_faces = 1)
mp_holistic = mp_sol.holistic

landmark_style = mp_drawing.DrawingSpec((0,255,0), thickness=1, circle_radius=1)
connection_style = mp_drawing.DrawingSpec((0,0,255), thickness=1, circle_radius=1)

Data

In [4]:
FACE = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 
        152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
LIPS = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 
        88, 95, 185, 40, 39, 37, 0, 267, 269, 270, 409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78]
LOWER_LIPS = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS = [185, 40, 39, 37, 0, 267, 269, 270, 409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 

# Left eyes indices 
LEFT_EYE = [362, 398, 384, 385, 386, 387, 388, 466, 382, 381, 380, 374, 373, 390, 249, 263]
LEFT_EYEBROW = [336, 296, 334, 293, 300, 276, 283, 282, 295, 285]

# Right eyes indices
RIGHT_EYE = [33, 246, 161, 160, 159, 158, 157, 173, 7, 163, 144, 145, 153, 154, 155, 133]  
RIGHT_EYEBROW = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46]

EYES = [362, 398, 384, 385, 386, 387, 388, 466, 382, 381, 380, 374, 373, 390, 249, 263, 
        33, 246, 161, 160, 159, 158, 157, 173, 7, 163, 144, 145, 153, 154, 155, 133] 

Functions

In [7]:
def eu_dist(pt1,pt2):
    x = np.square(pt2[0] - pt1[0])
    y = np.square(pt2[1] - pt1[1])
    z = np.square(pt2[2] - pt1[2])
    dist = np.sqrt(x + y + z)
    return(dist)


def blinked(le_nll, re_nll):
    left_v, right_v, ratio_le, ratio_re = 0, 0, 0, 0
    
    if len(le_nll) == 0 or len(re_nll) == 0:
        l_val, r_val = 0, 0
    
    else:
        # h for horizontal, v for vertical
        left_h = eu_dist(le_nll[0], le_nll[15])
        right_h = eu_dist(re_nll[0], re_nll[15])
        i = 1
        while i < 8:
            left_v += eu_dist(le_nll[i], le_nll[i + 7])
            right_v += eu_dist(re_nll[i], re_nll[i + 7])
            i += 1

        ratio_le = left_v/(7*left_h)
        ratio_re = right_v/(7*right_h)
        
        if ratio_le > 0.20:    # citation needed in value of EAR
            l_val = 1
        else:
            l_val = 0

        if ratio_re > 0.20:
            r_val = 1
        else:
            r_val = 0

    return((l_val, r_val))


def aplist(le_nll, re_nll):
    le, re = [],[]
    if len(le_nll) == 0 or len(re_nll) == 0:
        le, re = [], []
    else:
        le.append(le_nll[2])
        le.append(le_nll[14])
        le.append(le_nll[8])
        le.append(le_nll[9])
        le.append(le_nll[10])
        le.append(le_nll[11])
        le.append(le_nll[12])
        le.append(le_nll[15])
        le.append(le_nll[7])
        le.append(le_nll[6])
        le.append(le_nll[5])
        le.append(le_nll[4])
        le.append(le_nll[3])
        le.append(le_nll[13])
        le.append(le_nll[0])
        le.append(le_nll[1])

        re.append(re_nll[1])
        re.append(re_nll[15])
        re.append(re_nll[12])
        re.append(re_nll[11])
        re.append(re_nll[10])
        re.append(re_nll[9])
        re.append(re_nll[8])
        re.append(re_nll[14])
        re.append(re_nll[0])
        re.append(re_nll[13])
        re.append(re_nll[3])
        re.append(re_nll[4])
        re.append(re_nll[5])
        re.append(re_nll[6])
        re.append(re_nll[7])
        re.append(re_nll[2])
    
    return(le, re)

Driver Facial Expression Monitoring and Driver Pose Detection

In [8]:
cap = cv2.VideoCapture("eo1.mp4")

frame_no, drowsy = 0, 0
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        frame_no += 1
        success, frame = cap.read()
        frame = cv2.flip(frame, 1)
        ih, iw, ic = frame.shape
        le_nll, re_nll = [], []
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Make detection
        results_fm = facemesh.process(frame)
        results_hol = holistic.process(frame)
        
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results_fm.multi_face_landmarks:
            for flms in results_fm.multi_face_landmarks:
                # Eyes and Face Oval
                mp_drawing.draw_landmarks(frame, results_hol.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                          landmark_drawing_spec = None, connection_drawing_spec = mp_drawing_styles
                                          .get_default_face_mesh_contours_style())
                # Left Hand Landmarks
                mp_drawing.draw_landmarks(frame, results_hol.left_hand_landmarks, mp_sol.hands_connections.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
                                          mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
                # Right Hand Landmarks
                mp_drawing.draw_landmarks(frame, results_hol.right_hand_landmarks, mp_sol.hands_connections.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
                                          mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
                # Pose Landmarks
                mp_drawing.draw_landmarks(frame, results_hol.pose_landmarks, mp_sol.pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
                                          mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
                 
                
                for id, lms in enumerate(flms.landmark):
                    if id in LEFT_EYE:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        le_nll.append((x, y, z))                        
                        
                    elif id in RIGHT_EYE:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        re_nll.append((x, y, z))         
    
    
            status, instruct, color = "", "", (0,0,0)               
            a = aplist(le_nll, re_nll)
            a = blinked(a[0], a[1])

            if a[0] == 1 or a[1] == 1:
                status = "ALERT."
                color = (48, 255, 48)
                drowsy = 0
            
            else:
                drowsy += 1
                status = "BLINKED"
                color = (48, 255, 200)
                if drowsy > 10:
                    status = "DROWSY!!!"
                    color = (0,0,255)

                    
        cv2.putText(frame, str(frame_no), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (224, 224, 224), 3)
        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.imshow('MediaPipe', frame)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Release and destroy all windows
cap.release()
cv2.destroyAllWindows()